# Rebound model

Aim: Quantify the environmental impact due to the savings of households in consumption expenses, across different 
- industrial sectors and scenarios:
    - housing (rent): baseline for 2011, 
    - energy: efficient_devices, renewable_energy 
    - food-waste: avoidable_waste_saving
    - clothing: sufficiency, refuse, reshare, reuse for 2025  
    - furnishing: refuse, reuse for 2035 and 2050 
- temporal periods: years 2006-2017 
- spatial regions: parts of Switzerland


_Input_: The household budet survey files to train the data 

_Model_: A random forest or Artificial neural network model 

_Output_: The rebound expenses and environmental footprints of the households 

TOC<a id="toc"></a>

- <a href="#ini"> Step 0: Initialisation</a>
- <a href="#preprocess"> Step 1: Preprocessing</a>
- <a href="#model"> Step 2: Model </a>
- <a href="#post"> Step 3: Postprocessing </a>
- <a href="#lca"> Step 4: LCA </a>


Author: Rhythima Shinde, ETH Zurich

Co-Authors (for energy case study and temporal-regional rebound studies): Sidi Peng, Saloni Vijay, ETH Zurich

-------------------------------------------------------------------------------------------------------------------------------

## 0. Initialisation <a id = 'ini'></a>

<a href="#toc">back</a>

### 0.1. Input files & data parameters
- (1a) **seasonal_file** -> For the year 2009-11, the file is provided by <a href= https://pubs.acs.org/doi/full/10.1021/acs.est.8b01452>A.Froemelt</a>. It is modified based on original HBS(HABE) data that we <a href = https://www.bfs.admin.ch/bfs/en/home/statistics/economic-social-situation-population/surveys/hbs.html>obtain from Federal Statistical Office of Switzerland</a>. It is further modiefied in this code in the <a href='#preprocess'>preprocessing section</a> to rename columns.
- (1b) **seasonal_file_SI** -> Lists the HBS data columns and associated activities to calculate the consumption based environmental footprint. <a href=https://pubs.acs.org/doi/abs/10.1021/acs.est.8b01452>The file can be found here.</a>
- (2) **habe_month** -> the HBS household ids and their derivation to the month and year of the survey filled 
- (3) dependent_indices -> based on the HBS column indices, this file lists the relevant consumption expense parameters which are predicted 
- (4) **independent_indices** -> the HBS column indices which define the household socio-economic properties
- (5) **target_data** -> Selects the target dataset to predict the results. For most cases, it is the subset of the HBS (for the housing industry, it is the partner dataset 'ABZ', 'SCHL' or 'SM')  
- (6) **directory_name** -> based on the industry case, changes the dependent parameters, and income saved by the household (due to which the rebound is supposed to happen) - change the second value in the list. 

### 0.2. Model parameters
- (1) **iter_n** -> no.of iterations of runs
- (2) **model_name** -> Random Forest (RF) or ANN (Artificial Neural Network)

### 0.3. Analysis parameters
- (1) industry change: directory_name with following dependencies 
    - scenarios, 
    - partner_name/target dataset,
    - idx_column_savings_cons,
    - dependent_indices
- (2) year change: seasonal_file
    - specify which years (2006, 2007, 2008... 2017)
- (3) regional change: target_dataset
    - specify which regions (DE, IT, FR, ZH)
    - specify partner name (ABZ, SCHL, SM)

#### <p style='color:blue'>USER INPUT NEEDED: chose model settings, methods of preprocessing </p>

In [81]:
# model and folder settings
directory_name = 'housing' # 'housing' or 'furniture' or 'clothing' or 'energy'        
iter_n=1
model_name='RF' # 'RF' or 'ANN'

## preprocessing methods
option_deseason = 'deseasonal' # 'deseasonal' [option 1] or 'month-ind' [option 2]
if option_deseason ==  'month-ind':
    n_ind = 63
    independent_indices='raw_data/independent_month.csv' 
if option_deseason == 'deseasonal':
    n_ind = 39
    independent_indices='raw_data/independent.csv' 
input_normalise = 'no-normalise' #'no-normalise' for not normalising the data or 'normalise'

In [63]:
import pandas as pd
import numpy as np
import sklearn.multioutput as sko
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import scipy.stats as stats
import statistics
from sklearn.metrics import r2_score,mean_squared_error, explained_variance_score
from sklearn.model_selection import cross_val_score, KFold, train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import FunctionTransformer
import matplotlib.pyplot as plt
import brightway2
import seaborn as sns
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
import statsmodels.api as sm
from functools import reduce
import os
import pickle
import csv

# Additional libraries for neural network implementation 

# from numpy.random import seed
# seed(1)
# from tensorflow import set_random_seed
# set_random_seed(2)
# from keras import optimizers
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasRegressor

# Read the modified files by Nauser et al (2020)
# - HBS data (merged raw HBS files) "HABE_mergerd_2006_2017" 
# - tranlsation file 'HABE_Cname_translator.xlsx'
# - HBS hhids with the corresponding month of the survey 

###############################################################################################################################

seasonal_file = 'raw_data/HBS/HABE_merged_2006_2017.csv'
seasonal_file_SI = 'raw_data/HBS/HABE_Cname_translator.xlsx'
habe_month = 'raw_data/HBS/HABE_date.csv'
inf_index_file = 'raw_data/HBS/HABE_inflation_index_all.xlsx'
# seasonal_file = 'original_Andi_HBS/habe20092011_hh_prepared_imputed.csv' #based on the years
# seasonal_file_SI='original_Andi_HBS/Draft_Paper_8_v11_SupportingInformation.xlsx' 
# habe_month='original_Andi_HBS/habe_hh_month.csv'


## form the databases
df_habe = pd.read_csv(seasonal_file, delimiter=',', error_bad_lines=False, encoding='ISO-8859–1')
df_habe_month = pd.read_csv(habe_month, delimiter=',', error_bad_lines=False, encoding='ISO-8859–1')
inf_index = pd.read_excel(inf_index_file)

dependent_indices= 'raw_data/dependent_'+directory_name+'.csv'
dependent_indices_pd = pd.read_csv(dependent_indices, delimiter=',', encoding='ISO-8859–1')
dependent_indices_pd_name = pd.read_csv(dependent_indices,sep=',')["name"]
dependentsize=len(list(dependent_indices_pd_name))

independent_indices_pd = pd.read_csv(independent_indices, delimiter=',', encoding='ISO-8859–1')
list_independent_columns = pd.read_csv(independent_indices, delimiter=',', encoding='ISO-8859–1')['name'].to_list()
list_dependent_columns = pd.read_csv(dependent_indices, delimiter=',', encoding='ISO-8859–1')['name'].to_list()

In [4]:
#add more columns to perform temporal analysis (month_names and time_periods)

def label_month (row):
    if row['month'] == 1.0 :
        return 'January'
    if row['month'] == 2.0 :
        return 'February'
    if row['month'] == 3.0 :
        return 'March'
    if row['month'] == 4.0 :
        return 'April'
    if row['month'] == 5.0 :
        return 'May'
    if row['month'] == 6.0 :
        return 'June'
    if row['month'] == 7.0 :
        return 'July'
    if row['month'] == 8.0 :
        return 'August'
    if row['month'] == 9.0 :
        return 'September'
    if row['month'] == 10.0 :
        return 'October'
    if row['month'] == 11.0 :
        return 'November'
    if row['month'] == 12.0 :
        return 'December'
    
def label_period (row):
    if (row["year"] == 2006) or (row["year"] == 2007) or (row["year"] == 2008):
        return '1'
    if (row["year"] == 2009) or (row["year"] == 2010) or (row["year"] == 2011):
        return '2'
    if (row["year"] == 2012) or (row["year"] == 2013) or (row["year"] == 2014):
        return '3'
    if (row["year"] == 2015) or (row["year"] == 2016) or (row["year"] == 2017):
        return '4'
    
df_habe_month['month_name']=df_habe_month.apply(lambda row: label_month(row), axis=1)

df_habe_month['period']=df_habe_month.apply(lambda row: label_month(row), axis=1)


<p style = 'color:red'> TODO: update the right values for energy and food industry scenarios, and then merge in the above script </p>

In [5]:
if directory_name =='housing':
    scenarios = {'baseline_2011':500}
    target_data ='ABZ'
#     target_data = 'subset-HBS'
    idx_column_savings_cons = 'net_rent_and_mortgage_interest_of_principal_residence' #289  
    
if directory_name == 'furniture':
    scenarios = {'refuse_2035':17,'refuse_2050':17.4,'reuse_1_2035':6.9,
                           'reuse_1_2050':8.2,'reuse_2_2035':10.2,'reuse_2_2050':9.5}
    target_data = 'subset-HBS' 
    idx_column_savings_cons = 'furniture_and_furnishings,_carpets_and_other_floor_coverings_incl._repairs' #313  
    
if directory_name == 'clothing':
    scenarios = {'sufficiency_2025':76.08,'refuse_2025':5.7075,'share_2025':14.2875,'local_reuse_best_2025':9.13,
                         'local_reuse_worst_2025':4.54,'max_local_reuse_best_2025':10.25,'max_local_reuse_worst_2025':6.83}
    target_data = 'subset-HBS' 
    idx_column_savings_cons = 'clothing' #248     
    
if directory_name == 'energy':  
    scenarios = {'efficient_devices':30,'renewable_energy':300}
    target_data = 'subset-HBS' 
    idx_column_savings_cons = 'energy_of_principal_residence' #297 
    
if directory_name == 'food':  
    scenarios = {'avoidable_waste_saving':50}
    target_data = 'subset-HBS' 
    idx_column_savings_cons = 'food_and_non_alcoholic_beverages' #97

In [6]:
#functions to make relevant sector-wise directories

def make_pre_directory(outname,directory_name):
    outdir = 'preprocessing/'+directory_name
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname)
    return fullname

def make_pre_sub_directory(outname,directory_name,sub_dir):
    outdir = 'preprocessing/'+directory_name+'/'+sub_dir
    outdir1 = 'preprocessing/'+directory_name
    if not os.path.exists(outdir1):
        os.mkdir(outdir1)
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname)
    return fullname

def make_pre_sub_sub_directory(outname,directory_name,sub_dir,sub_sub_dir):
    outdir='preprocessing/'+directory_name+'/'+sub_dir+'/'+sub_sub_dir
    outdir1 = 'preprocessing/'+directory_name+'/'+sub_dir
    outdir2 = 'preprocessing/'+directory_name
    if not os.path.exists(outdir2):
        os.mkdir(outdir2)
    if not os.path.exists(outdir1):
        os.mkdir(outdir1)
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname)
    return fullname

## 1. Preprocessing <a id = 'preprocess' ></a>

TOC: <a id = 'toc-pre-pre'></a>
- <a href = #rename>1.1. Prepare training data</a>
- <a href = #deseasonal>1.2. Deseasonalise</a>
- <a href = #normal>1.3. Normalize</a>
- <a href = #check>1.4. Checks</a>

### 1.1. Prepare training data <a id='rename'></a>

<a href='#toc-pre'>back</a>

#### 1.1.1. Rename HBS columns

In [7]:
var_translate = pd.read_excel(seasonal_file_SI, sheet_name='translator', header=3, 
                              usecols=['habe_code', 'habe_eng_p', 'habe_eng', 'vcode', 'qcode'])
var_translate['habe_eng'] = var_translate['habe_eng'].str.strip()
var_translate['habe_eng'] = var_translate['habe_eng'].str.replace(' ', '_')
var_translate['habe_eng'] = var_translate['habe_eng'].str.replace('-', '_')
var_translate['habe_eng'] = var_translate['habe_eng'].str.replace('"', '')
var_translate['habe_eng'] = var_translate['habe_eng'].str.lower()
var_translate['habe_code'] = var_translate['habe_code'].str.lower()
dict_translate = dict(zip(var_translate['habe_code'], var_translate['habe_eng']))
df_habe.rename(columns=dict_translate, inplace=True)
dict_translate = dict(zip(var_translate['qcode'], var_translate['habe_eng']))
df_habe.rename(columns=dict_translate, inplace=True)
df_habe_rename = df_habe.loc[:, ~df_habe.columns.duplicated()]
pd.DataFrame.to_csv(df_habe_rename, 'preprocessing/0_habe_rename.csv', sep=',',index=False)

######## OLD RENAMING FUNCTIONS ###############

# def rename_columns():
#     df_rename = df_habe
#     df_rename = df_rename.rename(columns = rename_columns_SI('vcode'))
#     df_rename = df_rename.rename(columns=rename_columns_SI('qcode'))
#     df_rename.columns = df_rename.columns.str.replace(' ', '_')
#     df_rename.columns = df_rename.columns.str.replace('-', '_')
#     df_rename.columns = df_rename.columns.str.replace('"', '')
#     df_rename.columns = map(str.lower, df_rename.columns)

#     pd.DataFrame.to_csv(df_rename, 'preprocessing/0_habe_rename.csv', sep=',',index=False)
#     return df_rename


# def rename_columns_SI(code):
#     df_rename = df_habe
#     df_renaming_list = pd.read_excel(seasonal_file_SI, sheet_name='translator', header=3)
#     keys = list(df_renaming_list[code])
#     values = list(df_renaming_list['habe_eng_p'])
#     keys_new = list(df_rename.columns)
#     keys_index=[]
#     x=list(set(keys) & set(keys_new))
#     for y in x:
#         keys_index.append(keys.index(y))
#     keys_index = sorted(keys_index)
#     values_new = []
#     keys_new = []
#     for h in keys_index:
#         keys_new.append(keys[h])
#         values_new.append(values[h])
#     dictionary = dict(zip(keys_new, values_new))
#     return dictionary

# df_habe_rename = rename_columns()

#### 1.1.2. Inflation adjustment 

In [8]:
df_habe_rename = pd.read_csv('preprocessing/0_habe_rename.csv')
df_new = pd.merge(df_habe_rename, df_habe_month, on='haushaltid')
pd.DataFrame.to_csv(df_new,'preprocessing/0_habe_rename_month.csv', sep=',',index=False)

list_var_total = dependent_indices_pd_name.tolist()
list_var_total.pop()

# monetary variables inflation adjusted
list_mon = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
list_year = [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
list_var_total = list_var_total + ["disposable_income", "total_expenditures"]
# , 'infrequent_income'] 

df_inf = df_new

for col in list_var_total:
    for year in list_year:
        for mon in list_mon:
            df_inf.loc[(df_inf['year'] == year) & (df_inf['month'] == mon), col] = \
                df_inf.loc[(df_inf['year'] == year) & (df_inf['month'] == mon), col] / \
                inf_index.loc[(inf_index['year'] == year) & (inf_index['month'] == mon), col].values * 100

pd.DataFrame.to_csv(df_inf, 'preprocessing/1_habe_inflation.csv', sep=',', index=False, encoding='utf-8')

#### 1.1.3. Adapt the columns (optional - one hot encoding)

In [9]:
def new_columns(xx,directory_name):
    pd_df_saved = df_inf
    pd_df_saved.loc[:,'disposable_income'] = pd_df_saved['disposable_income'] - pd_df_saved.loc[:,xx] 
#     pd_df_saved['total_expenditures'] = pd_df_saved['total_expenditures'] - pd_df_saved.iloc[:,313]
    fullname = make_pre_directory('1_habe_rename_new_columns.csv',directory_name)
    pd.DataFrame.to_csv(pd_df_saved,fullname, sep=',',index=False)
    return pd_df_saved

df_habe_rename_saved = new_columns(idx_column_savings_cons,directory_name) # when redefining disposable income

#### 1.1.4. Remove outliers

In [10]:
def remove_outliers():
    df_outliers = df_habe_rename_saved  # TODO if using the new definition of disposable income: use the df_habe_rename_saved
#     df_outliers = df_outliers[np.abs(stats.zscore(df_outliers['disposable_income']))<10] 
#     df_outliers = df_outliers[np.abs(stats.zscore(df_outliers['saved_amount_(computed)']))<10]
    df_outliers = df_outliers[df_outliers['disposable_income'] >= 0] # simply keep all the 'sensible' disposable incomes
    # df_outliers = df_outliers[df_outliers['disposable_income'] <= 14800]  # ADDED CRITERIA FOR REMOVING OUTLIERS OF THE DISP_INCOME
    # df_outliers = df_outliers[df_outliers['total_expenditures'] >= 0]  # simply keep all the 'sensible' total_expenses
    df_outliers = df_outliers[df_outliers['saved_amount_(computed)'] >= 0]
    fullname = make_pre_directory('2_habe_rename_removeoutliers.csv',directory_name)
    pd.DataFrame.to_csv(df_outliers, fullname, sep=',', index=False)
    return df_outliers

df_habe_outliers = remove_outliers()

## aggregate the data as per the categories 
def accumulate_categories_habe(df,new_column,file_name):
    list_dependent_columns = pd.read_csv(dependent_indices, delimiter=',', encoding='ISO-8859–1')['name'].to_list()
    list_dependent_columns_new = list_dependent_columns
    list_dependent_columns_new.append('disposable_income')
    list_dependent_columns_new.append(new_column) # Might not always need this
    
    df = df[list_dependent_columns_new]
    df = df.loc[:,~df.columns.duplicated()] #drop duplicates
    
    df[new_column] = df.iloc[:, [17]]
    df['income'] = df.iloc[:, [16]]
    df['food'] = df.iloc[:,[0,1,2]].sum(axis=1)
    df['misc'] = df.iloc[:,[3,4]].sum(axis=1)
    df['housing'] = df.iloc[:, [5, 6]].sum(axis=1)
    df['services'] = df.iloc[:, [7,8,9]].sum(axis=1)
    df['travel'] = df.iloc[:, [10,11,12, 13, 14]].sum(axis=1)
    df['savings'] = df.iloc[:, [15]]    
    df = df[['income','food','misc','housing','services','travel','savings',new_column]]
    
    fullname = make_pre_directory(file_name,directory_name)
    pd.DataFrame.to_csv(df,fullname,sep=',',index= False)
    
    return df

df_outliers = pd.read_csv('preprocessing/'+directory_name+'/2_habe_rename_removeoutliers.csv')
df_habe_accumulate = accumulate_categories_habe(df_outliers,'month_name','2_habe_rename_removeoutliers_aggregated.csv')

### 1.2. Deasonalising <a id='deseasonal'></a>
- [Option 1] Clustering based on months 
- [Option 2] Use month and period as independent variable

<a href = #toc-pre-pre>back</a>

#### 1.2.1. [Option 1] Create monthly datasets,  Plots/ Tables / Statistical tests for HABE monthly data

In [11]:
if option_deseason == 'deseasonal' :
    
    def split_month():
        df_new = pd.read_csv('preprocessing/'+directory_name+'/2_habe_rename_removeoutliers.csv')
        df_month = df_new.groupby('month_name')

        for i in range(12):
            df_new_month=pd.DataFrame(list(df_month)[i][1])
            df_new_month['month_name']=df_new_month['month_name'].astype('str')
            fullname=make_pre_sub_directory('3_habe_monthly_'+df_new_month.month_name.unique()[0]+'.csv',
                                            directory_name,option_deseason)
            pd.DataFrame.to_csv(df_new_month,fullname,sep=',', index = False)

    split_month()

    # Split the accumulated categories per month

    def split_month_accumulated():
        df_new = pd.read_csv('preprocessing/'+directory_name+'/2_habe_rename_removeoutliers_aggregated.csv',sep=',')
        df_month = df_new.groupby('month_name')
        for i in range(12):
            df_new_month=pd.DataFrame(list(df_month)[i][1])
            df_new_month['month_name']=df_new_month['month_name'].astype('str')
            fullname = make_pre_sub_directory('3_habe_monthly_'+df_new_month.month_name.unique()[0]+'_aggregated.csv',
                                              directory_name,option_deseason)
            pd.DataFrame.to_csv(df_new_month,fullname, sep=',', index = False)

    split_month_accumulated()

#### 1.2.2. [Option 1] Making final clusters <a id ='finalclusters'></a>

<p style = 'color:blue'>USER INPUT NEEDED: edit the cluster-list below</p>

<p style = 'color:red'>TODO - join clusters based on the p-values calculated above directly</p>

In [12]:
## current clusters are made based on the mean table above

if option_deseason == 'deseasonal' :

    Cluster_month_lists  = {1:('January',),2:('February','March','April'),3:('May','June','July'),
                           4:('August','September','October','November'),5:('December',)}
    cluster_number_length = len(Cluster_month_lists)

    for key in Cluster_month_lists:
        df1=[]
        df_sum=[]
        for i in range(0,len(Cluster_month_lists[key])):
            print(Cluster_month_lists[key])
            df=pd.read_csv(make_pre_sub_directory('3_habe_monthly_{}'.format(Cluster_month_lists[key][i])+'.csv',
                                                  directory_name,option_deseason))
            df_sum.append(df.shape[0])
            df1.append(df)
        df_cluster = pd.concat(df1)
        assert df_cluster.shape[0]==sum(df_sum) # to check if the conacting was done correctly
        pd.DataFrame.to_csv(df_cluster,make_pre_sub_directory('4_habe_monthly_cluster_'+str(key)+'.csv',
                                                              directory_name,option_deseason),sep=',')

# TODO: update this to move to the sub directory of deseaspnal files
#     cluster_number_length = len(Cluster_month_lists)
#     for i in list(range(1,cluster_number_length+1)):
#         accumulate_categories_habe(df,'number_of_persons_per_household','4_habe_monthly_cluster_'+str(i)+'_aggregated.csv')

('January',)
('February', 'March', 'April')
('February', 'March', 'April')
('February', 'March', 'April')
('May', 'June', 'July')
('May', 'June', 'July')
('May', 'June', 'July')
('August', 'September', 'October', 'November')
('August', 'September', 'October', 'November')
('August', 'September', 'October', 'November')
('August', 'September', 'October', 'November')
('December',)


#### 1.2.3. Option 2: Month as independent variable

In [13]:
if option_deseason == 'month-ind' :
    cluster_number_length = 1
    
    # do one-hot encoding for month and year
    hbs_all = pd.read_csv('preprocessing/'+directory_name+'/1_habe_rename_new_columns.csv')
    month_encoding = pd.get_dummies(hbs_all.month_name, prefix='month')
    year_encoding = pd.get_dummies(hbs_all.year, prefix='year')
    hbs_all_encoding = pd.concat([hbs_all, month_encoding.reindex(month_encoding.index)], axis=1)
    hbs_all_encoding = pd.concat([hbs_all_encoding, year_encoding.reindex(year_encoding.index)], axis=1)
    
    for key in scenarios:
        output_encoding = make_pre_sub_sub_directory('3_habe_for_all_scenarios_encoding.csv',
                                                     directory_name,option_deseason,key)
        pd.DataFrame.to_csv(hbs_all_encoding,output_encoding,sep=',',index=False)
    
    month_name = month_encoding.columns.tolist()
    year_name = year_encoding.columns.tolist()

### 1.3. Normalisation  <a id='normal'></a>

<a href='#toc-pre'>back</a>

#### 1.3.1. Normalisation of HBS and target data

In [14]:
# ## NORMALISATION 

# if input_normalise == 'normalise':
#     def normalise_habe(cluster):
#         transformer = FunctionTransformer(np.log1p, validate=True)
        
#         if option_deseason == 'deseasonal':
#             df_deseasonal_file = pd.read_csv('preprocessing/'+directory_name+ '/' + option_deseason +
#                                              '/4_habe_monthly_cluster_'+str(cluster)+'.csv', 
#                                              delimiter=',')
#         if option_deseason == 'month-ind':
#             df_deseasonal_file = pd.read_csv('preprocessing/'+directory_name+ '/' + option_deseason +
#                                              '/3_habe_for_all_scenarios_encoding.csv',delimiter=',')
            
#         pd_df_new = df_deseasonal_file

#         for colsss in list_dependent_columns:
#             pd_df_new[[colsss]] = transformer.transform(df_deseasonal_file[[colsss]])

#         for colsss in list_independent_columns:
#             min_colsss = df_deseasonal_file[[colsss]].quantile([0.01]).values[0]
#             max_colsss = df_deseasonal_file[[colsss]].quantile([0.99]).values[0]
#             pd_df_new[[colsss]] = (df_deseasonal_file[[colsss]] - min_colsss) / (max_colsss - min_colsss)

#         pd_df = pd_df_new[list_independent_columns+['haushaltid']+list_dependent_columns]
#         pd_df = pd_df.fillna(0)
#         fullname = make_pre_directory('4_habe_deseasonal_'+str(cluster)+'_'+str(option_deseason)+'_normalised.csv',
#                                       directory_name)
#         pd.DataFrame.to_csv(pd_df,fullname,sep=',',index=False)

In [15]:
# if target_data == 'ABZ':
#     if input_normalise =='normalise':
#         def normalise_partner(i,key,option_deseason):
#             pd_df_partner = pd.read_csv('target_'+target_data+'.csv',delimiter=',')
#             df_complete =  pd.read_csv('preprocessing/'+directory_name+'/2_habe_rename_removeoutliers.csv',delimiter=',') 
#             pd_df_partner['disposable_income'] = pd_df_partner['disposable_income'] + i

#             for colsss in list_independent_columns:
#                 min_colsss = df_complete[[colsss]].quantile([0.01]).values[0]
#                 max_colsss = df_complete[[colsss]].quantile([0.99]).values[0]
#                 pd_df_partner[[colsss]] = (pd_df_partner[[colsss]] - min_colsss) / (max_colsss - min_colsss)

#             # pd_df_partner = pd_df_partner[pd_df_partner.iloc[:,30]<=1]
#             # pd_df_partner = pd_df_partner[pd_df_partner.iloc[:,32]<=1]
#             # pd_df_partner = pd_df_partner[pd_df_partner.iloc[:,33]>=0] #todo remove rows with normalisation over the range

#             fullname = make_pre_sub_sub_directory('5_final_'+ target_data + '_independent_final_'+str(i)+'.csv',
#                                               directory_name,option_deseason,key)

#             pd.DataFrame.to_csv(pd_df_partner,fullname,sep=',',index=False)
#             return pd_df_partner


#### 1.3.2. Preprocessing without normalisation

In [16]:
if input_normalise == 'no-normalise': 
    def normalise_habe(cluster):
        transformer = FunctionTransformer(np.log1p, validate=True)

        if option_deseason == 'deseasonal':
            df_deseasonal_file = pd.read_csv('preprocessing/'+directory_name+ '/' + option_deseason +
                                             '/4_habe_monthly_cluster_'+str(cluster)+'.csv', 
                                             delimiter=',')
        if option_deseason == 'month-ind':
            df_deseasonal_file = pd.read_csv('preprocessing/'+directory_name+ '/' + str(option_deseason) + '/' + str(key) +
                                             '/3_habe_for_all_scenarios_encoding.csv',delimiter=',')
           
        pd_df_new = df_deseasonal_file

        pd_df = pd_df_new[list_independent_columns+['haushaltid']+list_dependent_columns]
        pd_df = pd_df.fillna(0)
        fullname = make_pre_sub_directory('4_habe_deseasonal_'+str(cluster)+'_short.csv',
                                      directory_name,option_deseason)
        pd.DataFrame.to_csv(pd_df,fullname,sep=',',index=False)
        
for i in list(range(1,cluster_number_length+1)):
    df_normalise_habe_file = normalise_habe(i)

In [17]:
## Collecting the independent and dependent datasets

def truncate_all(key):
    if option_deseason == 'deseasonal':
        df_seasonal_normalised = pd.read_csv('preprocessing/'+directory_name+'/2_habe_rename_removeoutliers.csv', 
                                         delimiter=',', error_bad_lines=False)
    if option_deseason == 'month-ind':
        df_seasonal_normalised = pd.read_csv('preprocessing/'+directory_name+ '/' + str(option_deseason) + '/' + str(key) +
                                             '/3_habe_for_all_scenarios_encoding.csv',delimiter=',')
    
    df_habe_imputed_clustered_d = df_seasonal_normalised[list_dependent_columns]
    df_habe_imputed_clustered_i = df_seasonal_normalised[list_independent_columns]
    
    fullname_d = make_pre_sub_sub_directory('raw_dependent.csv',directory_name,option_deseason,key)
    fullname_in = make_pre_sub_sub_directory('raw_independent.csv',directory_name,option_deseason,key)
    
    pd.DataFrame.to_csv(df_habe_imputed_clustered_d,fullname_d,sep=',',index=False)
    pd.DataFrame.to_csv(df_habe_imputed_clustered_i,fullname_in,sep=',',index=False)

for key in scenarios:
    truncate_all(key)

In [18]:
## NORMALISATION 

if target_data == 'subset-HBS':
    def normalise_partner(i,key,option_deseason):
        N = 300 # TODO pass this as an argument when chosing subset of HBS
        pd_df_partner = pd.read_csv('preprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/raw_independent.csv', 
                                    delimiter=',', error_bad_lines=False)
        pd_df_partner = pd_df_partner.sample(frac=0.4, replace=True, random_state=1)
        pd_df_partner['disposable_income'] = pd_df_partner['disposable_income']+i
        fullname = make_pre_sub_sub_directory('5_final_'+ target_data + '_independent_final_'+str(i)+'.csv',
                                          directory_name,option_deseason,key)

        pd.DataFrame.to_csv(pd_df_partner,fullname,sep=',',index=False)
        return pd_df_partner
    

if target_data == 'ABZ':
    if input_normalise =='no-normalise':
        def normalise_partner(i,key,option_deseason):
            pd_df_partner = pd.read_csv('raw_data/target_'+target_data+'.csv',delimiter=',')
            df_complete =  pd.read_csv('preprocessing/'+directory_name+'/2_habe_rename_removeoutliers.csv',delimiter=',') 
            pd_df_partner['disposable_income'] = pd_df_partner['disposable_income'] - i

            # pd_df_partner = pd_df_partner[pd_df_partner.iloc[:,30]<=1]
            # pd_df_partner = pd_df_partner[pd_df_partner.iloc[:,32]<=1]
            # pd_df_partner = pd_df_partner[pd_df_partner.iloc[:,33]>=0] #todo remove rows with normalisation over the range

            fullname = make_pre_sub_sub_directory('5_final_'+ target_data + '_independent_final_'+str(i)+'.csv',
                                              directory_name,option_deseason,key)

            pd.DataFrame.to_csv(pd_df_partner,fullname,sep=',',index=False)
            return pd_df_partner

for key in scenarios:
    list_incomechange=[0,scenarios[key]]
    for i in list_incomechange:
        df_normalise_partner_file = normalise_partner(i,key,option_deseason)

### 1.4. Checks<a id='check'></a>

<a href='#toc-pre'>back</a>

In [19]:
# if input_normalise =='normalise':
#     def truncate(cluster_number):
#         if option_deseason == 'deseasonal':
#             df_seasonal_normalised = pd.read_csv('preprocessing/'+directory_name+ '/' + option_deseason +
#                                                  '/4_habe_deseasonal_'+str(cluster_number)+'_normalised.csv', 
#                                                  delimiter=',', error_bad_lines=False)
#         if option_deseason == 'month-ind':
#             df_seasonal_normalised = pd.read_csv('preprocessing/'+directory_name+ '/' + str(option_deseason) + '/' + str(key) +
#                                              '/3_habe_for_all_scenarios_encoding.csv',delimiter=',')
#         df_habe_imputed_clustered_d = df_seasonal_normalised[list_dependent_columns]
#         df_habe_imputed_clustered_dl = np.expm1(df_habe_imputed_clustered_d)
#         df_habe_imputed_clustered_i = df_seasonal_normalised[list_independent_columns]
        
#         fullname_dl = make_pre_sub_sub_directory('raw_dependent_old_'+str(cluster_number)+'.csv',directory_name,
#                                                  'checks',option_deseason)
#         fullname_d = make_pre_sub_sub_directory('raw_dependent_'+str(cluster_number)+'.csv',directory_name,
#                                                 'checks',option_deseason)
#         fullname_in = make_pre_sub_sub_directory('raw_independent_'+str(cluster_number)+'.csv',directory_name,
#                                                  'checks',option_deseason)
#         pd.DataFrame.to_csv(df_habe_imputed_clustered_dl,fullname_dl,sep=',',index=False)
#         pd.DataFrame.to_csv(df_habe_imputed_clustered_d,fullname_d,sep=',',index=False)
#         pd.DataFrame.to_csv(df_habe_imputed_clustered_i,fullname_in,sep=',',index=False)
    
if input_normalise =='no-normalise':
    def truncate(cluster_number):
        if option_deseason == 'deseasonal':
            df_seasonal_normalised = pd.read_csv('preprocessing/'+directory_name+ '/' + option_deseason +
                                                 '/4_habe_deseasonal_'+str(cluster_number)+'_short.csv', 
                                                 delimiter=',', error_bad_lines=False)
        if option_deseason == 'month-ind':
            df_seasonal_normalised = pd.read_csv('preprocessing/'+directory_name+ '/' + str(option_deseason) + '/' + str(key) +
                                             '/3_habe_for_all_scenarios_encoding.csv',delimiter=',')
        df_habe_imputed_clustered_d = df_seasonal_normalised[list_dependent_columns]
        df_habe_imputed_clustered_i = df_seasonal_normalised[list_independent_columns]
        
        fullname_d = make_pre_sub_sub_directory('raw_dependent_'+str(cluster_number)+'.csv',directory_name,
                                            'checks',option_deseason)
        fullname_in = make_pre_sub_sub_directory('raw_independent_'+str(cluster_number)+'.csv',directory_name,
                                             'checks',option_deseason)
        pd.DataFrame.to_csv(df_habe_imputed_clustered_d,fullname_d,sep=',',index=False)
        pd.DataFrame.to_csv(df_habe_imputed_clustered_i,fullname_in,sep=',',index=False)

In [20]:
for i in list(range(1,cluster_number_length+1)):
    truncate(i)

## 2. MODEL <a id = "model"></a>
    
<a href = "#toc">back</a>

TOC:<a id ='toc-model'></a>
- <a href = "#prep"> 2.1. Prepare train-test-target datasets</a>
- <a href = "#predict"> 2.2. Prediction</a>

### 2.1. Prepare train-test-target datasets <a id ='prep'></a>

<a href=#toc-model>back</a>

In [21]:
def to_haushalts(values,id_ix=0):
    haushalts = dict()
    haushalt_ids = np.unique(values[:,id_ix])
    for haushalt_id in haushalt_ids:
        selection = values[:, id_ix] == haushalt_id
        haushalts[haushalt_id] = values[selection]
    return haushalts

In [22]:
def split_train_test(haushalts,length_training,month_name,row_in_chunk):
    train, test = list(), list()
    cut_point = int(0.8*length_training)  # 0.9*9754 # declare cut_point as per the size of the imputed database #TODO check if this is too less
    print('Month/cluster and cut_point',month_name, cut_point)
    for k,rows in haushalts.items():
        train_rows = rows[rows[:,row_in_chunk] < cut_point, :]
        test_rows = rows[rows[:,row_in_chunk] > cut_point, :]
        train.append(train_rows[:, :])
        test.append(test_rows[:, :])
    return train, test

In [23]:
# ### NORMALISATION

# if input_normalise =='normalise':

#     def df_habe_train_test(df,month_name,length_training):
#         df=df.assign(id_split = list(range(df.shape[0])))
#         train, test = split_train_test(to_haushalts(df.values),length_training,month_name,row_in_chunk=df.shape[1]-1)

#         train_rows = np.array([row for rows in train for row in rows])
#         test_rows = np.array([row for rows in test for row in rows])

#         independent = list(range(0,independent_indices_pd.shape[0]))
#         dependent =  list(range(independent_indices_pd.shape[0]+1,
#                                 independent_indices_pd.shape[0]+dependent_indices_pd.shape[0]+1))

#         trained_independent = train_rows[:, independent]
#         trained_dependent = train_rows[:, dependent]
#         test_independent = test_rows[:, independent]
#         test_dependent = test_rows[:, dependent]

#         ## OPTIONAL lines FOR CHECK - comment if not needed
#         np.savetxt('preprocessing/'+directory_name+'/checks/'+option_deseason+'/trained_dependent_nonexp.csv', 
#                    trained_dependent, delimiter=',')    
#         np.savetxt('preprocessing/'+directory_name+'/checks/'+option_deseason+'/trained_dependent.csv', 
#                    np.expm1(trained_dependent),delimiter=',')
#         np.savetxt('preprocessing/'+directory_name+'/checks/'+option_deseason+'/trained_independent.csv', 
#                    trained_independent, delimiter=',')
#         np.savetxt('preprocessing/'+directory_name+'/checks/'+option_deseason+'/test_dependent.csv', 
#                    np.expm1(test_dependent), delimiter=',')
#         np.savetxt('preprocessing/'+directory_name+'/checks/'+option_deseason+'/test_independent.csv', 
#                    test_independent, delimiter=',')

#         return trained_independent,trained_dependent,test_independent,test_dependent

#     def df_partner_test(y):
#         df_partner = pd.read_csv('preprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/5_final_' + target_data + 
#                                  '_independent_final_' + str(y) + '.csv',delimiter=',')
#         length_training = df_partner.shape[0]
#         train_partner, test_partner = split_train_test(to_haushalts(df_partner.values),length_training,month_name,1) 
#         train_rows_partner = np.array([row for rows in train_partner for row in rows])
#         new_independent = list(range(0, n_ind)) # number of columns of the independent parameters
#         train_partner_independent = train_rows_partner[:, new_independent]

#         ### Optional lines for CHECK - comment if not needed
#         np.savetxt('preprocessing/'+directory_name+'/checks/'+option_deseason+'/train_partner_independent_' + model_name + '_' + str(y) + '.csv',
#                    train_partner_independent, delimiter=',')

#         return train_partner_independent

In [24]:
## form the train test datasets 

# NO-NORMALISATION
if input_normalise =='no-normalise':
    def df_habe_train_test(df,month_name,length_training):
        df=df.assign(id_split = list(range(df.shape[0])))
        train, test = split_train_test(to_haushalts(df.values),length_training,month_name,row_in_chunk=df.shape[1]-1)

        train_rows = np.array([row for rows in train for row in rows])
        test_rows = np.array([row for rows in test for row in rows])

        independent = list(range(0,independent_indices_pd.shape[0]))
        dependent =  list(range(independent_indices_pd.shape[0]+1,
                                independent_indices_pd.shape[0]+dependent_indices_pd.shape[0]+1))

        trained_independent = train_rows[:, independent]
        trained_dependent = train_rows[:, dependent]
        test_independent = test_rows[:, independent]
        test_dependent = test_rows[:, dependent]

        ## OPTIONAL lines FOR CHECK - comment if not needed
        # np.savetxt('raw/checks/trained_dependent_nonexp_'+str(month_name)+'.csv', trained_dependent, delimiter=',')    
        # np.savetxt('raw/checks/trained_independent_nonexp_'+str(month_name)+'.csv', trained_independent, delimiter=',')
        np.savetxt('preprocessing/'+directory_name+'/checks/'+option_deseason+'/test_dependent_'+str(month_name)+'.csv', 
                   test_dependent,delimiter=',')    
        np.savetxt('preprocessing/'+directory_name+'/checks/'+option_deseason+'/test_independent_'+str(month_name)+'.csv', 
                   test_independent, delimiter=',')

        return trained_independent,trained_dependent,test_independent,test_dependent
    
    def df_partner_test(y):
        df_partner = pd.read_csv('preprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/5_final_' + target_data + 
                                 '_independent_final_' + str(y) + '.csv', delimiter=',')
        length_training = df_partner.shape[0]
        train_partner, test_partner = split_train_test(to_haushalts(df_partner.values),
                                                       length_training,cluster_number,1) 
        train_rows_partner = np.array([row for rows in train_partner for row in rows])
        new_independent = list(range(0, n_ind))
        train_partner_independent = train_rows_partner[:, new_independent]

        ### Optional lines for CHECK - comment if not needed
        np.savetxt('preprocessing/'+directory_name+'/checks/'+option_deseason+'/train_partner_independent_' + 
                   model_name + '_' + str(y) + '.csv', train_partner_independent, delimiter=',')

        return train_partner_independent

In [25]:
def make_post_directory(outname,directory_name):
    outdir = 'postprocessing/'+directory_name
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname)
    return fullname

def make_post_sub_directory(outname,directory_name,sub_dir):
    outdir_1='postprocessing/'+directory_name
    if not os.path.exists(outdir_1):
        os.mkdir(outdir_1)
    outdir = 'postprocessing/'+directory_name+'/'+sub_dir
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname)
    return fullname

def make_post_sub_sub_directory(outname,directory_name,sub_dir,sub_sub_dir):
    outdir_1='postprocessing/'+directory_name
    if not os.path.exists(outdir_1):
        os.mkdir(outdir_1)
    outdir = 'postprocessing/'+directory_name+'/'+sub_dir
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    outdir_2='postprocessing/'+directory_name+'/'+sub_dir+'/'+sub_sub_dir
    if not os.path.exists(outdir_2):
        os.mkdir(outdir_2)
    fullname = os.path.join(outdir_2, outname)
    return fullname

In [26]:
# FOR NO NORMALISATION AND TEST DATA 

# def df_test(y,cluster_number):
#     pd_df_partner = pd.read_csv('raw/checks/trained_independent_'+str(cluster_number)+'.csv', delimiter=',', header = None)
#     pd_df_partner.iloc[:,-1] = pd_df_partner.iloc[:,-1] + y

#     pd.DataFrame.to_csv(pd_df_partner, 'raw/checks/5_trained_independent_'+str(cluster_number)+'_'+str(y)+'.csv', 
#                         sep=',',index=False)
#     return pd_df_partner

# def df_stratified_test(y):
#     pd_df_partner = pd.read_csv('raw/checks/5_setstratified_independent_1_'+str(y)+'.csv', delimiter=',')
#     return pd_df_partner

In [27]:
#If using Neural Networks

# def ANN():
#     nn = Sequential()
#     nn.add(Dense(39,kernel_initializer='normal',activation="relu",input_shape=(39,)))
#     nn.add(Dense(50,kernel_initializer='normal',activation="relu"))
#     nn.add(Dense(100,kernel_initializer='normal',activation="relu"))
#     nn.add(Dense(100,kernel_initializer='normal',activation="relu") )
#     # nn.add(Dense(100,kernel_initializer='normal',activation="relu"))
#     # nn.add(Dense(100,kernel_initializer='normal',activation="relu"))
#     nn.add(Dense(dependentsize,kernel_initializer='normal')) #,kernel_constraint=min_max_norm(min_value=0.01,max_value=0.05)))
#     sgd = optimizers.SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True)
#     nn.compile(optimizer=sgd, loss='mean_squared_error', metrics=['accuracy'])
#     return nn

### 2.2. Clustered Prediction <a id='predict'></a>

<a href='#toc-model'>back</a>

In [28]:
# ## NORMALISATION

# if input_normalise =='normalise':

#     def fit_predict_cluster(i,y,cluster_number,key):
#         df = pd.read_csv('preprocessing/'+directory_name+'/'+option_deseason+
#                          '/4_habe_deseasonal_'+str(cluster_number)+'_normalised.csv',
#                          delimiter=',',error_bad_lines=False, encoding='ISO-8859–1')
#         length_training = df.shape[0]
#         trained_independent, trained_dependent, test_independent, test_dependent = df_habe_train_test(df,
#                                                                                                       str(cluster_number),
#                                                                                                       length_training)
#         train_partner_independent = df_partner_test(y)
        
#         if model_name == 'ANN':
#             estimator = KerasRegressor(build_fn=ANN)
#             estimator.fit(trained_independent, trained_dependent, epochs=100, batch_size=5, verbose=0)

#             ### PREDICTION FROM HERE
#             prediction_nn = estimator.predict(train_partner_independent)
#             prediction_nn_denormalised = np.expm1(prediction_nn)
#             fullname = make_post_sub_sub_directory('predicted_' + model_name + '_' + str(y) + '_' + str(i) 
#                        + '_' + str(cluster_number) + '.csv',directory_name,option_deseason,key)
#             np.savetxt(fullname, prediction_nn_denormalised, delimiter=',')

#             ### TEST PREDICTION
#             prediction_nn_test = estimator.predict(test_independent)
#             prediction_nn_test_denormalised = np.expm1(prediction_nn_test)
#             fullname = make_post_sub_sub_directory('predicted_test' + model_name + '_' + str(y) + '_' + str(i) 
#                        + '_' + str(cluster_number) + '.csv',directory_name,option_deseason,key)
#             np.savetxt(fullname, prediction_nn_test_denormalised, delimiter=',')

#             ### CROSS VALIDATION FROM HERE
#             kfold = KFold(n_splits=10, random_state=12, shuffle=True)
#             results1 = cross_val_score(estimator, test_independent, test_dependent, cv=kfold)
#             print("Results_test: %.2f (%.2f)" % (results1.mean(), results1.std()))

#         if model_name == 'RF':
#             estimator = sko.MultiOutputRegressor(RandomForestRegressor(n_estimators=100, max_features=n_ind, random_state=30))
#             estimator.fit(trained_independent, trained_dependent)

#             ### PREDICTION FROM HERE
#             prediction_nn = estimator.predict(train_partner_independent)
#             results0 = estimator.oob_score
#             prediction_nn_denormalised = np.expm1(prediction_nn)
#             fullname = make_post_sub_sub_directory('predicted_' + model_name + '_' + str(y) + '_' + str(i) 
#                        + '_' + str(cluster_number) + '.csv',directory_name,option_deseason,key)
#             np.savetxt(fullname, prediction_nn_denormalised, delimiter=',')

#              ### TEST PREDICTION
#             prediction_nn_test = estimator.predict(test_independent)
#             prediction_nn_test_denormalised = np.expm1(prediction_nn_test)
#             fullname = make_post_sub_sub_directory('predicted_test' + model_name + '_' + str(y) + '_' + str(i) 
#                        + '_' + str(cluster_number) + '.csv',directory_name,option_deseason,key)
#             np.savetxt(fullname, prediction_nn_test_denormalised, delimiter=',')        

#             #### CROSS VALIDATION FROM HERE
#             kfold = KFold(n_splits=10, random_state=12, shuffle=True)
#             # results0 = estimator.oob_score
#             # results1 = cross_val_score(estimator, test_independent, test_dependent, cv=kfold)
#             results2 = r2_score(test_dependent,prediction_nn_test)
#             results3 = mean_squared_error(test_dependent,prediction_nn_test)
#             results4 = explained_variance_score(test_dependent,prediction_nn_test)
#             # print("cross_val_score: %.2f (%.2f)" % (results1.mean(), results1.std()))
#             # print("oob_r2_score: %.2f " % results0)
#             print("r2_score: %.2f " % results2)
#             print("mean_squared_error: %.2f " % results3)
#             print("explained_variance_score: %.2f " % results4)


In [84]:
### FOR NO NORMALISATION

if input_normalise =='no-normalise':
    def fit_predict_cluster(i,y,cluster_number,key):
        df_non_normalised = pd.read_csv('preprocessing/'+directory_name+'/'+option_deseason+'/4_habe_deseasonal_'+
                                        str(cluster_number)+ '_short.csv', delimiter=',',
                                        error_bad_lines=False, encoding='ISO-8859–1')
        length_training = df_non_normalised.shape[0]
        print(length_training)
        trained_independent, trained_dependent, test_independent, test_dependent = df_habe_train_test(df_non_normalised,
                                                                                                      str(cluster_number),
                                                                                                      length_training)
        train_partner_independent = df_partner_test(y)
        
        ### Additional for the HBS test data subset
        # test_new_independent = df_test(y,1) # chosing just one cluster here
        # sratified_independent = df_stratified_test(y)
    
        if model_name == 'ANN':
            estimator = KerasRegressor(build_fn=ANN)
            estimator.fit(trained_independent, trained_dependent, epochs=100, batch_size=5, verbose=0)

            ### PREDICTION FROM HERE
            prediction_nn = estimator.predict(train_partner_independent)
            fullname = make_post_sub_sub_directory('predicted_' + model_name + '_' + str(y) + '_' + str(i) 
                       + '_' + str(cluster_number) +'.csv',directory_name,option_deseason,key)
            np.savetxt(fullname, prediction_nn, delimiter=',')

            ### TEST PREDICTION
            prediction_nn_test = estimator.predict(test_independent)
            fullname = make_post_sub_sub_directory('predicted_test_' + model_name + '_' + str(y) + '_' + str(i) 
                       + '_' + str(cluster_number) +'.csv',directory_name,option_deseason,key)
            np.savetxt(fullname, prediction_nn_test, delimiter=',')

            ### CROSS VALIDATION FROM HERE
            kfold = KFold(n_splits=10, random_state=12, shuffle=True)
            results1 = cross_val_score(estimator, test_independent, test_dependent, cv=kfold)
            print("Results_test: %.2f (%.2f)" % (results1.mean(), results1.std()))

        if model_name == 'RF':
            estimator = sko.MultiOutputRegressor(RandomForestRegressor(n_estimators=100, max_features=n_ind, random_state=30))
            estimator.fit(trained_independent, trained_dependent)
            
            ### FEATURE IMPORTANCE
            rf = RandomForestRegressor()
            rf.fit(trained_independent, trained_dependent)
            FI = rf.feature_importances_
            list_independent_columns = pd.read_csv(independent_indices, delimiter=',', encoding='ISO-8859–1')['name'].to_list()
            independent_columns = pd.DataFrame(list_independent_columns)
            FI_names = pd.DataFrame(FI)
            FI_names = pd.concat([independent_columns, FI_names], axis=1)
            FI_names.columns = ['independent_variables', 'FI_score']
            pd.DataFrame.to_csv(FI_names,'preprocessing/'+directory_name+'/8_habe_feature_importance'+ '_' +
                                str(y) + '_' + str(i) + '_' + str(cluster_number) +'.csv', sep=',',index= False)
            FI_names_sorted = FI_names.sort_values('FI_score', ascending = False)
#             print(FI_names_sorted)

            ### PREDICTION FROM HERE
            prediction_nn = estimator.predict(train_partner_independent)
            fullname = make_post_sub_sub_directory('predicted_' + model_name + '_' + str(y) + '_' + str(i) 
                       + '_' + str(cluster_number) +'.csv',directory_name,option_deseason,key)
            np.savetxt(fullname, prediction_nn, delimiter=',')

             ### TEST PREDICTION
            prediction_nn_test = estimator.predict(test_independent)
            fullname = make_post_sub_sub_directory('predicted_test_' + model_name + '_' + str(y) + '_' + str(i) 
                       + '_' + str(cluster_number) +'.csv',directory_name,option_deseason,key)
            np.savetxt(fullname, prediction_nn_test, delimiter=',')     

            #### CROSS VALIDATION FROM HERE
            kfold = KFold(n_splits=10, random_state=12, shuffle=True)
            
            for i in range(16):
                column_predict = pd.DataFrame(test_dependent).iloc[:,i]
                model = sm.OLS(column_predict, test_independent).fit() 
                print(i)
                print('standard error=',model.bse) 
            
            # results0 = estimator.oob_score
            # results1 = cross_val_score(estimator, test_independent, test_dependent, cv=kfold)
#             results2 = r2_score(test_dependent,prediction_nn_test)
#             results3 = mean_squared_error(test_dependent,prediction_nn_test)
#             results4 = explained_variance_score(test_dependent,prediction_nn_test)
            # print("cross_val_score: %.2f (%.2f)" % (results1.mean(), results1.std()))
            # print("oob_r2_score: %.2f " % results0)
            print("r2_score: %.2f " % results2)
            print("mean_squared_error: %.2f " % results3)
            print("explained_variance_score: %.2f " % results4)

In [85]:
# CLUSTER of MONTHS - PREDICTIONS
for cluster_number in list(range(1,cluster_number_length+1)):
    print(cluster_number)
    for j in range(0, iter_n):
        for key in scenarios:
            list_incomechange=[0,scenarios[key]]
            for y in list_incomechange:
                fit_predict_cluster(j,y,cluster_number,key)

1
2301
Month/cluster and cut_point 1 1840
Month/cluster and cut_point 1 1199
0
standard error= x1     40.490810
x2     62.180617
x3     56.538947
x4     59.404511
x5     61.127747
x6     63.129861
x7     59.727916
x8     57.480385
x9     10.596318
x10     8.655813
x11    27.756329
x12    44.666962
x13    22.514507
x14    46.116155
x15    31.469952
x16    45.371581
x17    40.587932
x18    44.957662
x19    42.205714
x20    40.076987
x21    51.184074
x22    70.868329
x23    39.386659
x24    34.649143
x25    44.974733
x26    42.571748
x27    41.757918
x28    42.678597
x29    40.474273
x30    52.930873
x31    75.118096
x32    16.421303
x33    32.867710
x34    12.805623
x35    34.176171
x36    28.871578
x37    20.073164
x38    44.995352
x39     0.003324
dtype: float64
1
standard error= x1     26.571955
x2     40.805816
x3     37.103489
x4     38.984006
x5     40.114874
x6     41.428754
x7     39.196239
x8     37.721305
x9      6.953798
x10     5.680347
x11    18.214996
x12    29.312540
x13  

NameError: name 'results2' is not defined


## 3.POSTPROCESSING <a id = "post"></a>
    
<a href="#toc">back</a>

### 3.1. Average of the clustered predictions

In [32]:

if option_deseason == 'month-ind': 
    df_habe_outliers = pd.read_csv('preprocessing/'+directory_name+'/'+option_deseason+'/4_habe_deseasonal_'+
                                            str(cluster_number)+ '_short.csv', delimiter=',')
if option_deseason == 'deseasonal':
    df_habe_outliers = pd.read_csv('preprocessing/'+directory_name+'/2_habe_rename_removeoutliers.csv', delimiter=',')

In [33]:
model_name = 'RF'

In [34]:
def average_pandas_cluster(y,cluster_number,key):
    df_all = []
    df_trained_partner = pd.read_csv('preprocessing/'+directory_name+'/checks/'+option_deseason+'/train_partner_independent_'+
                                     model_name+'_'+str(y)+'.csv')
    for i in range(0,iter_n):
        df = pd.read_csv('postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_' + model_name + '_' + 
                         str(y) + '_' + str(i) + '_' + 
                         str(cluster_number) + '.csv', delimiter = ',', header=None)
        df_all.append(df)
    glued = pd.concat(df_all, axis=1, keys=list(map(chr,range(97,97+iter_n))))
    glued = glued.swaplevel(0, 1, axis=1)
    glued = glued.groupby(level=0, axis=1).mean()
    glued_new = glued.reindex(columns=df_all[0].columns)

    max_income = df_habe_outliers[['disposable_income']].quantile([0.99]).values[0]
    min_income = df_habe_outliers[['disposable_income']].quantile([0.01]).values[0]
    glued_new['income'] = df_trained_partner[df_trained_partner.columns[-1]]
    pd.DataFrame.to_csv(glued_new, 'postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_' + model_name + '_' + str(y)
                        + '_'+str(cluster_number)+'.csv', sep=',',header=None,index=False)

In [35]:
for key in scenarios:
    list_incomechange=[0,scenarios[key]]
    for y in list_incomechange:
        for cluster_number in list(range(1,cluster_number_length+1)):
            average_pandas_cluster(y,cluster_number,key)

In [36]:
def accumulate_categories_cluster(y,cluster_number):
    df_income = pd.read_csv('postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_' + model_name + '_' + str(y)
                            + '_'+str(cluster_number)+'.csv', 
                            sep=',',header=None)
#     df_income['household_size'] = df_income.iloc[:, [17]]
    df_income['income'] = df_income.iloc[:, [16]]
    df_income['food'] = df_income.iloc[:,[0,1,2]].sum(axis=1)
    df_income['misc'] = df_income.iloc[:,[3,4]].sum(axis=1)
    df_income['housing'] = df_income.iloc[:, [5, 6]].sum(axis=1)
    df_income['services'] = df_income.iloc[:, [7, 8, 9 ]].sum(axis=1)
    df_income['travel'] = df_income.iloc[:, [10, 11, 12, 13, 14]].sum(axis=1)
    df_income['savings'] = df_income.iloc[:, [15]]             
    df_income = df_income[['income','food','misc','housing','services','travel','savings']]
    pd.DataFrame.to_csv(df_income,
                        'postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_' + model_name + '_' + str(y) 
                        + '_'+str(cluster_number)+'_aggregated.csv', sep=',',index=False)
    return df_income

In [37]:
for key in scenarios:
    list_incomechange=[0,scenarios[key]]
    for y in list_incomechange: 
        for cluster_number in list(range(1,cluster_number_length+1)):
            accumulate_categories_cluster(y,cluster_number)

In [38]:
# aggregation of clusters

list_dfs_month=[]
for key in scenarios:
    list_incomechange=[0,scenarios[key]]
    for y in list_incomechange:    
        for cluster_number in list(range(1,cluster_number_length+1)):
            pd_predicted_month = pd.read_csv('postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_' + model_name + '_' + str(y) 
                                             + '_'+str(cluster_number)+'_aggregated.csv', delimiter = ',')
            list_dfs_month.append(pd_predicted_month)

        df_concat = pd.concat(list_dfs_month,sort=False)

        by_row_index = df_concat.groupby(df_concat.index)
        df_means = by_row_index.mean()
        pd.DataFrame.to_csv(df_means,'postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_' + model_name + '_' + str(y) + '_' + 
                            str(dependentsize) +'_aggregated.csv', sep=',',index=False)

### 3.2. Calculate differences/ rebounds

In [51]:
list_dependent_columns = pd.read_csv(dependent_indices, delimiter=',', encoding='ISO-8859–1')['name'].to_list()

def difference_new():
    for cluster_number in list(range(1,cluster_number_length+1)):
        for key in scenarios:
            list_incomechange=[0,scenarios[key]]
            for i in range(0,iter_n):
                df_trained_partner = pd.read_csv('preprocessing/'+directory_name+'/checks/'+'/'+option_deseason+'/train_partner_independent_'+
                                     model_name+'_'+str(y)+'.csv',header=None)
                df_500 = pd.read_csv('postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_' + model_name + '_'
                                     +str(list_incomechange[1])+ '_'+str(i)
                                     + '_'+str(cluster_number)+'.csv', delimiter=',',header=None)
                df_0 = pd.read_csv('postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_' + model_name + '_0_' 
                                    + str(i)  + '_'+str(cluster_number)+ '.csv', delimiter=',',header=None)
                df_500.columns = list_dependent_columns
                df_0.columns = df_500.columns
                df_diff = -df_500+df_0
                if option_deseason == 'month-ind':
                    df_diff['disposable_income']=df_trained_partner[df_trained_partner.columns[-25]]
                if option_deseason == 'deseasonal':
                    df_diff['disposable_income']=df_trained_partner[df_trained_partner.columns[-1]]
                pd.DataFrame.to_csv(df_diff,'postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_' + model_name 
                                    + '_rebound_'+str(i)+ '_' + str(cluster_number) + '.csv',sep=',',index=False)

In [52]:
difference_new()

In [53]:
def average_clusters(key):
    df_all = []
    for i in range(0,iter_n):
        df = pd.read_csv('postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_'+ model_name + '_rebound_' + 
                         str(i)+ '_' + str(cluster_number)+'.csv',delimiter=',',index_col=None)
        df_all.append(df)
    
    df_concat = pd.concat(df_all,sort=False)

    by_row_index = df_concat.groupby(df_concat.index)
    df_means = by_row_index.mean()
    pd.DataFrame.to_csv(df_means, 'postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_'+model_name +'_rebound.csv',
                        sep=',',index=False)

In [54]:
for key in scenarios:
    average_clusters(key)

In [55]:
def accumulate_categories(key):
    df_income = pd.read_csv('postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_'+model_name+ '_rebound.csv',delimiter=',')
#     df_income['household_size'] = df_income.iloc[:, [17]]
    df_income['income'] = df_income.iloc[:, [16]]
    df_income['food'] = df_income.iloc[:,[0,1,2]].sum(axis=1)
    df_income['misc'] = df_income.iloc[:,[3,4]].sum(axis=1)
    df_income['housing'] = df_income.iloc[:, [5, 6]].sum(axis=1)
    df_income['services'] = df_income.iloc[:, [7, 8, 9]].sum(axis=1)
    df_income['travel'] = df_income.iloc[:, [10, 11, 12,13, 14]].sum(axis=1)
    df_income['savings'] = df_income.iloc[:, [15]]    
    df_income = df_income[['income','food','misc','housing','services','travel','savings']]#'transfers','total_sum'
    data[key]=list(df_income.mean())
    if list(scenarios.keys()).index(key) == len(scenarios)-1:
        df = pd.DataFrame(data, columns = [key for key in scenarios],
                  index=['income','food','misc','housing','services','travel','savings'])
        print(df)
        pd.DataFrame.to_csv(df.T, 'postprocessing/rebound_results_'+directory_name+ '_income.csv', sep=',',index=True)
    pd.DataFrame.to_csv(df_income,
                        'postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_'+model_name+ '_rebound_aggregated.csv',
                        sep=',',index=False)

In [56]:
data={}
for key in scenarios:
    accumulate_categories(key)

          baseline_2011
income      4589.453636
food          74.577799
misc          42.708161
housing       64.318477
services      11.483421
travel       109.763693
savings      184.740454


In [57]:
groups=('<2000','2000-4000','4000-6000','6000-8000','8000-10000','>10000')
def income_group(row):
    if row['disposable_income'] <= 2000:
        return groups[0]
    if row['disposable_income'] <= 4000:
        return groups[1]
    if row['disposable_income'] <= 6000:
        return groups[2]
    if row['disposable_income'] <= 8000:
        return groups[3]
    if row['disposable_income'] <= 10000:
        return groups[4]
    if row['disposable_income'] > 10000:
        return groups[5]

In [58]:
def accumulate_income_groups():
    df_income = pd.read_csv('postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_'+model_name+ '_rebound.csv',
                            delimiter=',')
    df_income['income_group'] = df_income.apply(lambda row: income_group(row), axis=1)
    df_income_new = df_income.groupby(['income_group']).mean()
    pd.DataFrame.to_csv(df_income_new,'postprocessing/rebound_results_'+directory_name+ '_income_categories.csv', sep=',',index=True)
    pd.DataFrame.to_csv(df_income,
                        'postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_'+model_name+ '_rebound_income.csv',
                        sep=',',index=False)

In [59]:
accumulate_income_groups()

In [60]:
groups=('<2000','2000-4000','4000-6000','6000-8000','8000-10000','>10000')
def income_group(row):
    if row['income'] <= 2000 :
        return groups[0]
    if row['income'] <= 4000:
        return groups[1]
    if row['income'] <= 6000:
        return groups[2]
    if row['income'] <= 8000:
        return groups[3]
    if row['income'] <= 10000:
        return groups[4]
    if row['income'] > 10000:
        return groups[5]

In [61]:
def accumulate_income_groups_new():
    df_income = pd.read_csv('postprocessing/'+directory_name+'/'+option_deseason+'/'+key+'/predicted_'+model_name+ '_rebound_aggregated.csv',
                            delimiter=',')
    print(df_income.columns)
    df_income['income_group'] = df_income.apply(lambda row: income_group(row), axis=1)
    df_income_new = df_income.groupby(['income_group']).mean()
    pd.DataFrame.to_csv(df_income_new,'postprocessing/rebound_results_'+directory_name+ '_categories.csv', sep=',',index=True)

In [62]:
accumulate_income_groups_new()

Index(['income', 'food', 'misc', 'housing', 'services', 'travel', 'savings'], dtype='object')


## 4. LCA <a id = "lca"></a>

<a href = '#toc'>back</a>

1. Make a file with associated impacts_per_FU for each HABE category: 
    - a. Get the ecoinvent data from brightway
    - b. Get the exiobase data from direct file (Livia's)
    - c. Attach the heia and Agribalyse values 
2. Convert the impact_per_FU to impact_per_expenses
3. Run the following scripts to  
    - (a) allocate the income category to each household in HBS (train data) and ABZ (target data) 
    - (b) calculate environmental impact per consumption main-category per income group as listed in the raw/dependent_10.csv
        - (1) From HBS: % of expense of consumption sub-category per consumption main-category as listed in the raw/dependent_10.csv 
        - (2) expenses per FU of each consumption sub-category 
    - (c) From target data: Multiply the rebound results (consumption expenses) with the env. impact values above 
        based on the income of the household
    
    OR 
    
    Use A.Kim's analysis here: https://github.com/aleksandra-kim/consumption_model for the calculation of impacts_per_FU for each HABE catergory


In [ ]:
import pickle
import csv
file = open('LCA/contribution_scores_sectors_allfu1.pickle','rb')
x = pickle.load(file)
print(x)
with open('LCA/impacts_per_FU_sectors.csv', 'w') as output:
    writer = csv.writer(output)
    for key, value in x:
        writer.writerow([key, value])

In [ ]:
import pickle
import csv

In [ ]:
import pickle
import csv
file = open('LCA/contribution_scores_5categories_allfu1.pickle','rb')
x = pickle.load(file)
print(x)
with open('LCA/impacts_per_FU.csv', 'w') as output:
    writer = csv.writer(output)
    for key, value in x.items():
        writer.writerow([key, value])

In [ ]:
file = open('LCA/contribution_scores_v2.pickle','rb')
x1 = pickle.load(file)
with open('LCA/impacts_per_FU.csv', 'w') as output:
    writer = csv.writer(output)
    for key, value in x1.items():
        writer.writerow([key, value])

In [ ]:
file = open('LCA/contribution_scores_sectors_allfu1.pickle','rb')
x = pickle.load(file)
with open('LCA/impacts_per_FU_sectors.csv', 'w') as output:
    writer = csv.writer(output)
    for key, value in x.items():
        writer.writerow([key, value])

In [ ]:
import pandas as pd
## TODO use the manually updated CHF/FU to calculate the income per expense 
df_expense = pd.read_csv('LCA/impacts_per_expense.csv',sep=',',index_col='sector')
df_income_CHF = pd.read_csv('postprocessing/rebound_results_'+directory_name+ '_income.csv',sep=',')
for i in ['food','travel','housing','food','misc','services']:
    df_income_CHF[i+'_GHG']=df_expense.loc[i,'Average of GWP/CHF']*df_income_CHF[i]
    pd.DataFrame.to_csv(df_income_CHF,'postprocessing/rebound_results_'+directory_name+ '_income_all_GHG.csv',sep=',')

In [ ]:
##################### USETOX Analysis for MAJA #########################
file = open('LCA/USEtox/contribution_scores_sectors_usetox.pickle','rb')
x1 = pickle.load(file)
with open('LCA/impacts_per_FU_usetox.csv', 'w') as output:
    writer = csv.writer(output)
    for key, value in x1.items():
        writer.writerow([key, value])
x1